In [1]:
!pip install tensorflow-model-optimization
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 9.2 MB/s eta 0:00:00


In [2]:
# Load the TF Flowers dataset
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
num_classes = info.features['label'].num_classes

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0  # Normalize
    return image, label

# Prepare training and validation datasets
train_data = dataset['train'].map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_data = dataset['train'].map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
# Define a simple CNN model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the baseline model
model.fit(train_data, validation_data=val_data, epochs=5)

9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/5
115/115 [==============================] - 37s 89ms/step - loss: 0.6968 - accuracy: 0.7428 - val_loss: 0.4217 - val_accuracy: 0.8520
Epoch 2/5
115/115 [==============================] - 12s 83ms/step - loss: 0.3743 - accuracy: 0.8689 - val_loss: 0.3128 - val_accuracy: 0.8970
Epoch 3/5
115/115 [==============================] - 11s 75ms/step - loss: 0.3039 - accuracy: 0.8975 - val_loss: 0.2513 - val_accuracy: 0.9180
Epoch 4/5
115/115 [==============================] - 11s 76ms/step - loss: 0.2552 - accuracy: 0.9147 - val_loss: 0.2286 - val_accuracy: 0.9248
Epoch 5/5
115/115 [==============================] - 11s 77ms/step - loss: 0.2247 - accuracy: 0.9262 - val_loss: 0.2091 - val_accuracy: 0.9376


In [5]:
# Apply gradient-based pruning
def apply_gradient_based_pruning(model):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    # Define pruning parameters with gradient-based selection
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.2,
                                                                  final_sparsity=0.5,
                                                                  begin_step=0,
                                                                  end_step=1000)
    }

    pruned_model = prune_low_magnitude(model, **pruning_params)
    return pruned_model

# Apply pruning and recompile
pruned_model = apply_gradient_based_pruning(model)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fine-tune pruned model
# Include the UpdatePruningStep callback during training
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]
pruned_model.fit(train_data, validation_data=val_data, epochs=5, callbacks=callbacks)

Epoch 1/5
115/115 [==============================] - 63s 183ms/step - loss: 0.2930 - accuracy: 0.9052 - val_loss: 0.5505 - val_accuracy: 0.7932
Epoch 2/5
115/115 [==============================] - 15s 118ms/step - loss: 0.4738 - accuracy: 0.8316 - val_loss: 0.5317 - val_accuracy: 0.8044
Epoch 3/5
115/115 [==============================] - 16s 117ms/step - loss: 0.5653 - accuracy: 0.7962 - val_loss: 0.5550 - val_accuracy: 0.8008
Epoch 4/5
115/115 [==============================] - 15s 111ms/step - loss: 0.6678 - accuracy: 0.7602 - val_loss: 0.6443 - val_accuracy: 0.7561
Epoch 5/5
115/115 [==============================] - 15s 117ms/step - loss: 0.7476 - accuracy: 0.7177 - val_loss: 0.6998 - val_accuracy: 0.7330


In [6]:
# Evaluate pruned model
_, pruned_acc = pruned_model.evaluate(val_data)
print(f"Pruned Model Accuracy: {pruned_acc:.4f}")

115/115 [==============================] - 5s 41ms/step - loss: 0.6998 - accuracy: 0.7330
Pruned Model Accuracy: 0.7330
